In [1]:
import pandas as pd
import numpy as np
import pickle
import sys
sys.path.append("../")
import src.Resources as src

import warnings
warnings.filterwarnings('ignore')


In this notebook i will clean all the data from imdb. We start with this base csv called Movies.

In [2]:
main_df = pd.read_csv('../Data/Movies.csv', index_col= 0)  

We change the name of each of the collumns

In [3]:
main_df.columns = ["Movie_name","Movie_img","Movie_genre","Movie_description","Movie_rating","Casting","Movie_votes","Movie_directors","Movie_stars"]

We run two cleaning functions from Resources.py

In [4]:
main_df["Movie_rating"] = main_df["Movie_rating"].apply(lambda x: src.movie_rating_cleaner(x))
main_df["Movie_votes"] = main_df["Movie_votes"].apply(lambda x: src.movie_vote_cleaner(x))

And now it's time to run the main cleaning function for imdb, it also grabs all the .pkl we have in data and concat them together to make one DataFrame of them all.

In [5]:
clean_df = src.movie_cleaner(main_df)

Here we need to reset the index, drop duplicates and reset index again.

In [6]:
clean_df.reset_index(inplace=True)
clean_df.drop("index",axis=1,inplace=True)
clean_df.drop_duplicates(inplace=True)
clean_df.reset_index(inplace=True)
clean_df.drop("index",axis=1,inplace=True)

Here we divide our Directors and Stars by the character "|", this will be useful later on for my recommendation.

In [7]:
clean_df["Movie_stars"] = clean_df["Movie_stars"].apply(lambda x: src.split_casting(x))
clean_df["Movie_directors"] = clean_df["Movie_directors"].apply(lambda x: src.split_casting(x))

I don't need casting anymore so we will drop it.

In [8]:
clean_df.drop("Casting",axis=1,inplace=True)

We make sure that there aren't any null values, and then we save it as Movies_clean

In [9]:
clean_df["Movie_directors"]

0          John Francis Daley| Jonathan Goldstein 
1                                  Chad Stahelski 
2                                 Jeremy Garelick 
3                                   James Cameron 
4                                  Kyle Newacheck 
                            ...                   
130080             Jessica Kitrick| Lewis Lovhaug 
130081                              Tristan Price 
130082                              Aman Sachdeva 
130083                            Riccardo Ghione 
130084                               Edward Conna 
Name: Movie_directors, Length: 130085, dtype: object

In [10]:
clean_df.isnull().sum()

Movie_name           0
Movie_img            0
Movie_genre          0
Movie_description    0
Movie_rating         0
Movie_votes          0
Movie_directors      0
Movie_stars          0
dtype: int64

In [11]:
clean_df.to_csv("../Data/Movies_clean.csv")

Now we will clean the Goodreads scraped data

In this notebook I will clean the data scraped from Goodreads and as well put it all in one DataFrame.
First, we start with the DataFrame Books as a baseline.

In [12]:
book_df = pd.read_csv('../Data/Books.csv', index_col= 0)  

In [13]:
book_df.head(1)

,Book_author,Book_img,Book_description,Book_rating,Book_votes,Book_title,Book_genre
0,William Shakespeare,https://images-na.ssl-images-amazon.com/images...,"In Romeo and Juliet, Shakespeare creates a vio...",3.74,2462752,Romeo and Juliet,Classics|Plays|Fiction|Romance


This is a list of all the .pkl that we want to load to our DataFrame, i chose them all, if in a future you would scrap new books you need to put the name of the pkl in here aswell, keep in mind to remove Books from the name here.

Exemple: Books_Action.pkl for this list would be just Action.

In [14]:
genres= ["Action","Adventure","Comedy","Crime","Drama","Fantasy","History","Horror","Mystery","Romance","Sciencefiction","Superhero","Thriller","List"]

Here we run the cleaning function, this function aswell puts all the pickles together into one DataFrame.

In [15]:
book_df = src.book_cleaner(genres,book_df)

We reset the index, drop duplicates, reset index again and then we multiply the rating by 2 so its in the same scale as imdb.

In [16]:
book_df.reset_index(inplace=True)
book_df.drop("index",axis=1,inplace=True)
book_df.drop_duplicates(inplace=True)
book_df.reset_index(inplace=True)
book_df.drop("index",axis=1,inplace=True)
book_df["Book_rating"] = book_df["Book_rating"].apply(lambda x: src.book_rating_multiplier(x))

We make sure that there aren't any Nan values, in this case there is one so ill just replace it for: "No description".

In [17]:
book_df.isnull().sum()

Book_author         0
Book_img            0
Book_description    1
Book_rating         0
Book_votes          0
Book_title          0
Book_genre          0
dtype: int64

In [18]:
book_df=book_df.fillna("No description")

This is the look of the DataFrame clean and with all of the new data we got from the pickles that were scraped with our goodreads selenium function.

And finally we save it as Books_clean

In [19]:
book_df.to_csv("../Data/Books_clean.csv")

Here we load Books_clean to simplify the genres.

In [22]:
book_df2 = pd.read_csv('../Data/Books_clean.csv', index_col= 0)  

This function uses a dictionary in my library.py to simplify all the genres of the books, then this is applied with a map.

In [23]:
book_df2['Book_genre'] = book_df['Book_genre'].apply(src.map_genre)

These are now the new genres for the books.

In [24]:
book_df2

,Book_author,Book_img,Book_description,Book_rating,Book_votes,Book_title,Book_genre
0,William Shakespeare,https://images-na.ssl-images-amazon.com/images...,"In Romeo and Juliet, Shakespeare creates a vio...",7.48,2462752,Romeo and Juliet,Romance
1,William Shakespeare,https://images-na.ssl-images-amazon.com/images...,"Among Shakespeare's plays, ""Hamlet"" is conside...",8.04,888492,Hamlet,Drama
2,William Shakespeare,https://images-na.ssl-images-amazon.com/images...,"One night on the heath, the brave and respecte...",7.80,836710,Macbeth,Family
3,William Shakespeare,https://images-na.ssl-images-amazon.com/images...,"In Othello, Shakespeare creates a powerful dra...",7.78,368863,Othello,Drama
4,William Shakespeare,https://images-na.ssl-images-amazon.com/images...,Shakespeare's intertwined love polygons begin ...,7.90,513517,A Midsummer Night's Dream,Fantasy
...,...,...,...,...,...,...,...
18043,Alice Schroeder,https://images-na.ssl-images-amazon.com/images...,The personally revealing and complete biograph...,8.26,48538,The Snowball: Warren Buffett and the Business ...,History|Biography
18044,Jonathan Lethem,https://images-na.ssl-images-amazon.com/images...,Lionel Essrog is Brooklyn’s very own self-appo...,7.72,39729,Motherless Brooklyn,Mystery|Crime
18045,Ruth Rendell,https://images-na.ssl-images-amazon.com/images...,Dazzling psychological suspense. Razor-sharp d...,7.36,13305,From Doon With Death,Mystery|Crime
18046,Shawn Achor,https://images-na.ssl-images-amazon.com/images...,Our most commonly held formula for success is ...,8.28,35581,The Happiness Advantage: The Seven Principles ...,Biography|History|Documentary


We drop any Nan because that means that it doesn't coincide with any genre that i have for my movies, so I won't be able to do a good recommendation based in the genre.

In [25]:
book_df2.isna().sum()

Book_author          0
Book_img             0
Book_description    11
Book_rating          0
Book_votes           0
Book_title           0
Book_genre          49
dtype: int64

In [26]:
book_df2.dropna(inplace=True)

In [27]:
book_df2

,Book_author,Book_img,Book_description,Book_rating,Book_votes,Book_title,Book_genre
0,William Shakespeare,https://images-na.ssl-images-amazon.com/images...,"In Romeo and Juliet, Shakespeare creates a vio...",7.48,2462752,Romeo and Juliet,Romance
1,William Shakespeare,https://images-na.ssl-images-amazon.com/images...,"Among Shakespeare's plays, ""Hamlet"" is conside...",8.04,888492,Hamlet,Drama
2,William Shakespeare,https://images-na.ssl-images-amazon.com/images...,"One night on the heath, the brave and respecte...",7.80,836710,Macbeth,Family
3,William Shakespeare,https://images-na.ssl-images-amazon.com/images...,"In Othello, Shakespeare creates a powerful dra...",7.78,368863,Othello,Drama
4,William Shakespeare,https://images-na.ssl-images-amazon.com/images...,Shakespeare's intertwined love polygons begin ...,7.90,513517,A Midsummer Night's Dream,Fantasy
...,...,...,...,...,...,...,...
18043,Alice Schroeder,https://images-na.ssl-images-amazon.com/images...,The personally revealing and complete biograph...,8.26,48538,The Snowball: Warren Buffett and the Business ...,History|Biography
18044,Jonathan Lethem,https://images-na.ssl-images-amazon.com/images...,Lionel Essrog is Brooklyn’s very own self-appo...,7.72,39729,Motherless Brooklyn,Mystery|Crime
18045,Ruth Rendell,https://images-na.ssl-images-amazon.com/images...,Dazzling psychological suspense. Razor-sharp d...,7.36,13305,From Doon With Death,Mystery|Crime
18046,Shawn Achor,https://images-na.ssl-images-amazon.com/images...,Our most commonly held formula for success is ...,8.28,35581,The Happiness Advantage: The Seven Principles ...,Biography|History|Documentary


In [28]:
book_df2.to_csv("../Data/Books_clean_filtered.csv")

Now we do the cleaning to prepare the recomendation

In [29]:
movies = pd.read_csv("../Data/Movies_clean.csv",index_col=0)
books = pd.read_csv("../Data/Books_clean_filtered.csv",index_col=0)

We rename our columns

In [30]:
movies.columns = ["Title","Image","Genre","Description","Rating","Votes","Directors","Stars"]
books.columns = ["Author","Image","Description","Rating","Votes","Title","Genre"]

Drop unnecesary columns

In [31]:
movies_rec = movies.drop(["Directors","Stars"],axis=1)
books_rec = books.drop(["Author",],axis=1)
books_rec = books_rec.reindex(columns=["Title","Image","Genre","Description","Rating","Votes"])

We create a column with a identifier

In [32]:
books_rec["Type"] = "Book"
movies_rec["Type"] = "Movie"

We concat both movies and books

In [33]:
recomendation = pd.concat([books_rec,movies_rec])

We create a title identifier, there may be duplicates if a book has a movie adaptation

In [34]:
recomendation["Title_id"] = recomendation["Title"]+"_"+recomendation["Type"]

We filter to have only titles with over 5000 votes

In [35]:
recomendation = recomendation[recomendation["Votes"]>5000]

In [36]:
recomendation.drop_duplicates(inplace=True)
recomendation = recomendation.drop_duplicates(subset="Title_id")
recomendation = recomendation.reset_index()
recomendation = recomendation.drop("index",axis=1)

In [37]:
recomendation

,Title,Image,Genre,Description,Rating,Votes,Type,Title_id
0,Romeo and Juliet,https://images-na.ssl-images-amazon.com/images...,Romance,"In Romeo and Juliet, Shakespeare creates a vio...",7.48,2462752,Book,Romeo and Juliet_Book
1,Hamlet,https://images-na.ssl-images-amazon.com/images...,Drama,"Among Shakespeare's plays, ""Hamlet"" is conside...",8.04,888492,Book,Hamlet_Book
2,Macbeth,https://images-na.ssl-images-amazon.com/images...,Family,"One night on the heath, the brave and respecte...",7.80,836710,Book,Macbeth_Book
3,Othello,https://images-na.ssl-images-amazon.com/images...,Drama,"In Othello, Shakespeare creates a powerful dra...",7.78,368863,Book,Othello_Book
4,A Midsummer Night's Dream,https://images-na.ssl-images-amazon.com/images...,Fantasy,Shakespeare's intertwined love polygons begin ...,7.90,513517,Book,A Midsummer Night's Dream_Book
...,...,...,...,...,...,...,...,...
24561,El gendarme y los extraterrestres,https://m.media-amazon.com/images/S/sash/4Fyxw...,Comedy|Crime|Sci-Fi,"In St. Tropez, French gendarme Cruchot and his...",6.20,8890,Movie,El gendarme y los extraterrestres_Movie
24562,The Capture,https://m.media-amazon.com/images/S/sash/4Fyxw...,Sci-Fi|Thriller,A group of scientists discover a soul guide an...,5.00,5442,Movie,The Capture_Movie
24563,Hunter Prey,https://m.media-amazon.com/images/S/sash/4Fyxw...,Sci-Fi|Thriller,A spaceship crashes on a desert planet with 3 ...,5.80,6114,Movie,Hunter Prey_Movie
24564,Infection,https://m.media-amazon.com/images/S/sash/4Fyxw...,Drama|Horror|Sci-Fi,"INFECTION takes place in a dark, isolated hosp...",6.00,5614,Movie,Infection_Movie


Now we reorder our columns and export to csv

In [38]:
recomendation = recomendation.reindex(columns=["Image","Title","Rating","Votes","Genre","Description","Type","Title_id"])

In [39]:
recomendation.to_csv("../Data/recomendation.csv")